# Recurrent Neural Networks

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.contrib import rnn

## Build class for TS prediction

We are working with multivariate input series and univariate output series. **input_dim** is dimension of single point. The output_dim is 1.

In [14]:
class SeriesPredictor:
    
    def __init__(self, input_dim, seq_size, hidden_dim=10):
        self.input_dim = input_dim
        self.seq_size = seq_size
        self.hidden_dim = hidden_dim
        self.x = tf.placeholder(tf.float32, [None, seq_size, input_dim])
        self.y = tf.placeholder(tf.float32, [None, seq_size])
        self.W_out = tf.Variable(tf.random_normal([hidden_dim, 1]), name='W_out')
        self.b_out = tf.Variable(tf.random_normal([1]), name='b_out')
        
        self.cost = tf.reduce_mean(tf.square(self.model() - self.y))
        self.train_op = tf.train.AdamOptimizer().minimize(self.cost)
        
        self.saver = tf.train.Saver()
        
        
    def model(self):
        cell = rnn.BasicLSTMCell(self.hidden_dim)
        print(self.x)
        outputs, states = tf.nn.dynamic_rnn(cell, self.x, dtype=tf.float32)
        num_examples = tf.shape(self.x)[0]
        W_repeated = tf.tile(tf.expand_dims(self.W_out, 0), [num_examples, 1, 1])
        return tf.matmul(outputs, W_repeated) + self.b_out
        
        
    def train(self, train_x, train_y):
        with tf.Session() as sess:
            tf.get_variable_scope().reuse_variables()
            sess.run(tf.global_variables_initializer())
            for i in range(10):
                _, mse = sess.run([self.train_op, self.cost], feed_dict={self.x: train_x, self.y: train_y})
                if i % 100 == 0:
                    print(i, mse)
            self.saver.save(sess, '../models/rnn1.ckpt')
    
    
    def test(self, test_x):
        return 0

## Test

In [15]:
predictor = SeriesPredictor(input_dim=1, seq_size=4, hidden_dim=10)
train_x = [[[1], [2], [5], [6]],
           [[5], [7], [7], [9]],
           [[3], [4], [5], [7]]]
train_y = [[1, 3, 7, 11],
           [5, 12, 14, 15],
           [3, 7, 9, 12]]

predictor.train(train_x, train_y)

Tensor("Placeholder_10:0", shape=(?, 4, 1), dtype=float32)


ValueError: Attempt to have a second RNNCell use the weights of a variable scope that already has weights: 'rnn/basic_lstm_cell'; and the cell was not constructed as BasicLSTMCell(..., reuse=True).  To share the weights of an RNNCell, simply reuse it in your second calculation, or create a new one with the argument reuse=True.